In [1]:
#Step 1: Merging the Two Text Files (Train and Test Sets)
import pandas as pd

In [2]:
train_df = pd.read_csv('train_English_Data_Complete_FakeNews.txt', delimiter='\t')

In [3]:
test_df = pd.read_csv('test_English_Data_Complete_FakeNews.txt', delimiter='\t')

In [4]:
# Merge train and test DataFrames
merged_df = pd.concat([train_df, test_df], ignore_index=True)

In [5]:
# Save the merged dataset to a new text file or CSV
merged_df.to_csv('merged_dataset.csv', index=False)

In [6]:
merged_df.head()


,Domain,Topic,News,Label
0,Sports,Roger Federer beats Frances Tiafoe on return ...,Roger Federer beat American teenager Frances T...,Legit
1,Celebrity,'It was a long time coming': Heidi Klum ends s...,Seal and Heidi Klum insisted their split was '...,Fake
2,Celebrity,Liam Payne Just Dissed Harry Styles' Solo Music,"Harry Styles' new music isn't for everyone, no...",Fake
3,Entertainment,We all know why the right is angry at Tomi Lahren,Conservative darling-turned-pariah Tomi Lahren...,Legit
4,Celebrity,Inside Beyoncé's baby shower,Don't act like you didn't know Beyoncé was goi...,Legit


Step 2: Build an AI Model for Fake News Detection

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix


In [8]:
# Load the merged dataset
df = pd.read_csv('merged_dataset.csv')


In [9]:
# Drop any rows with missing values (optional)
df.dropna(subset=['News', 'Label'], inplace=True)


In [10]:
# Split the dataset into training and testing sets
X = df['News']  # Features (news articles)
y = df['Label']  # Labels (0: Legit, 1: Fake)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
# Drop rows where 'text' column is NaN
X_train = X_train.dropna()
X_test = X_test.dropna()

In [12]:
# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [13]:
# Initialize and train the Logistic Regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)


LogisticRegression()

In [14]:
# Make predictions on the test set
y_pred = model.predict(X_test_tfidf)


In [15]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

In [16]:
print(f"Model Accuracy: {accuracy*100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)

Model Accuracy: 46.43%
Confusion Matrix:
[[43 60]
 [45 48]]


# Using Random Forest as LogisticRegression model is not so accurate.

In [17]:
from sklearn.ensemble import RandomForestClassifier


In [18]:
# Initialize Random Forest model
rf_model = RandomForestClassifier(random_state=42)


In [19]:
# Fit the model to the training data
rf_model.fit(X_train_tfidf, y_train)


RandomForestClassifier(random_state=42)

In [20]:
# Make predictions
y_pred_rf = rf_model.predict(X_test_tfidf)

In [21]:
# Evaluate performance
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

Random Forest Accuracy: 0.5561224489795918


#XGBoost Classifier
!pip install xgboost

#Because my labels y_train contain the strings 'Fake' and 'Legit', while XGBoost expects numerical labels like 0 and 1. I need to convert my categorical labels ('Fake' and 'Legit') into numerical labels before training the model by using Label Encoder.

In [22]:
from sklearn.preprocessing import LabelEncoder

In [23]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

In [24]:
!pip install xgboost

In [25]:
import xgboost as xgb

In [26]:
# Initialize XGBoost model
xgb_model = xgb.XGBClassifier(random_state=42)

#Because my labels y_train contain the strings 'Fake' and 'Legit', while XGBoost expects numerical labels like 0 and 1. I need to convert my categorical labels ('Fake' and 'Legit') into numerical labels before training the model by using Label Encoder.

In [27]:
#from sklearn.preprocessing import LabelEncoder


In [28]:
# Initialize LabelEncoder
#label_encoder = LabelEncoder()

In [29]:
# Fit and transform the labels in y_train and y_test
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [33]:
# Fit the model to the training data
xgb_model.fit(X_train_tfidf, y_train)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1], got ['Fake' 'Legit']